# DCGAN implemented on MxNet Gluon

In [4]:
from __future__ import print_function
import os
import mxnet as mx
import random
from mxnet import nd, autograd
from mxnet import gluon
import numpy as np
import matplotlib.pyplot as plt

## Initialize the hyper parameters

In [5]:
class Options:
    def __init__(self):
        self.dataset = 'cifar' # cifar10 | lsun | imagenet | folder | lfw
        #self.dataroot = '/EBS100G/GAN_resutls/lfw/lfw-deepfunneled' # path to dataset
        self.dataroot = './data' # path to dataset
        self.workers = 2 # number of data loading workers
        self.batchSize = 64 # input batch size
        self.imageSize = 64 # the height / width of the input image to network'
        self.nz =100 # size of the latent z vector
        self.ngf = 64
        self.ndf = 64 
        self.nc = 3 #numb color
        self.niter =125 # number of epochs to train for
        self.lr = 0.01 # learning rate, default=0.0002
        self.beta1 = 0.5 # beta1 for adam. default=0.5
        self.ctx = mx.gpu() #  enables gpu
        self.ngpu = 1 # number of GPUs to use
        self.G_net = '' # path to netG (to continue training)
        self.D_net = '' #help="path to netD (to continue training)")
        self.outf = './data/mxnet_cifar' # help='folder to output images and model checkpoints')
        self.manualSeed = random.randint(1, 10000) # manual seed 

opt = Options()
try:
    os.makedirs(opt.outf)
except OSError:
    pass
mx.random.seed(opt.manualSeed)



## Load the target data

In [6]:
def transformer(data, label):
    data = mx.image.imresize(data, opt.imageSize, opt.imageSize)
    data = mx.nd.transpose(data, (2,0,1))
    data = data.astype(np.float32)/128-1
    return data, label

## Creat a data iterator

In [9]:
train_data = gluon.data.DataLoader(
    gluon.data.vision.CIFAR10(opt.dataroot, train=True, transform=transformer),
    batch_size= opt.batchSize, shuffle=True, last_batch='discard')

test_data = gluon.data.DataLoader(
    gluon.data.vision.CIFAR10(opt.dataroot, train=False, transform=transformer),
    batch_size=opt.batchSize, shuffle=False, last_batch='discard')

## Creat the Generator and Discriminator networks

In [ ]:
G_Net = gluon.nn.Sequential()
with G_Net.name_scope():
    #first layer
    G_Net.add(gluon.nn.Conv2DTranspose(channels=opt.ngf * 8, kernel_size=4,use_bias=False))
    G_Net.add(gluon.nn.BatchNorm(axis = 1, momentum = 0.1,center=True))
    G_Net.add(gluon.nn.Activation("relu"))
    #second layer
    G_Net.add(gluon.nn.Conv2DTranspose(channels=opt.ngf * 4, kernel_size=4,strides = 2,padding=1,use_bias=False))
    G_Net.add(gluon.nn.BatchNorm(axis = 1, momentum = 0.1,center=True))
    G_Net.add(gluon.nn.Activation("relu"))
    #tird layer
    G_Net.add(gluon.nn.Conv2DTranspose(channels=opt.ngf * 2, kernel_size=4,strides = 2,padding=1,use_bias=False))
    G_Net.add(gluon.nn.BatchNorm(axis = 1, momentum = 0.1,center=True))
    G_Net.add(gluon.nn.Activation("relu"))
    #fourth layer
    G_Net.add(gluon.nn.Conv2DTranspose(channels=opt.ngf, kernel_size=4,strides = 2,padding=1,use_bias=False))
    G_Net.add(gluon.nn.BatchNorm(axis = 1, momentum = 0.1,center=True))
    G_Net.add(gluon.nn.Activation("relu"))
    #fifth layer
    G_Net.add(gluon.nn.Conv2DTranspose(channels=opt.nc, kernel_size=4,strides = 2,padding=1,use_bias=False))
    G_Net.add(gluon.nn.Activation("tanh"))

    
    
D_Net = gluon.nn.Sequential()
with D_Net.name_scope():
    #first layer
    D_Net.add(gluon.nn.Conv2D(channels=opt.ndf , kernel_size=4,strides = 2,padding=1, use_bias=False))
    D_Net.add(gluon.nn.LeakyReLU(0.2))
    #second layer
    D_Net.add(gluon.nn.Conv2D(channels=opt.ndf * 2, kernel_size=4,strides = 2,padding=1,use_bias=False))
    D_Net.add(gluon.nn.BatchNorm(axis = 1, momentum = 0.1,center=True))
    D_Net.add(gluon.nn.LeakyReLU(0.2))
    #tird layer
    D_Net.add(gluon.nn.Conv2D(channels=opt.ndf * 4, kernel_size=4,strides = 2,padding=1,use_bias=False))
    D_Net.add(gluon.nn.BatchNorm(axis = 1, momentum = 0.1,center=True))
    D_Net.add(gluon.nn.LeakyReLU(0.2))
    #fourth layer
    D_Net.add(gluon.nn.Conv2D(channels=opt.ndf * 8, kernel_size=4,strides = 2,padding=1,use_bias=False))
    D_Net.add(gluon.nn.BatchNorm(axis = 1, momentum = 0.1,center=True))
    D_Net.add(gluon.nn.LeakyReLU(0.2))
    #fifth layer
    D_Net.add(gluon.nn.Conv2D(channels=opt.ndf * 8, kernel_size=4,strides = 2,padding=0,use_bias=False))
    D_Net.add(gluon.nn.Dense(1,activation ='sigmoid'))
    
g_net = G_Net
d_net = D_Net

## A function to visualizing the created and real images

In [ ]:
def image_show(data,padding = 2):
    import math
    datanp = (data.asnumpy().transpose((0, 2, 3, 1))+1.0)*128
    x_dim = min(8, opt.batchSize)
    y_dim = int(math.ceil(float(opt.batchSize) / x_dim))
    height, width = int(opt.imageSize + padding), int(opt.imageSize + padding)
    grid = np.zeros(( height * y_dim + 1 + padding // 2, width * x_dim + 1 + padding // 2,3))
    k = 0
    for y in range(y_dim):
        for x in range(x_dim):
            if k >= opt.batchSize:
                break
            start_y = y * height + 1 + padding // 2
            end_y = start_y + height - padding 
            start_x = x * width + 1 + padding // 2
            end_x = start_x + width - padding
            np.copyto(grid[start_y:end_y,start_x:end_x],datanp[k])
            k = k + 1
    plt.figure(figsize=(10,10))
    plt.axis('off')
    plt.imshow(grid)
    plt.show()
def binary_cross_entropy(yhat, y):
    return - (y * nd.log(yhat)+(1 - y ) * nd.log(1 - yhat))

## Initialize the networks and the optimizers

In [ ]:
#Initialization

g_net.collect_params().initialize(mx.init.Xavier(magnitude=0.02), ctx=opt.ctx)
d_net.collect_params().initialize(mx.init.Xavier(magnitude=0.02), ctx=opt.ctx)
G_trainer = gluon.Trainer(g_net.collect_params(), 'Adam', {'learning_rate': opt.lr * 100,'beta1':opt.beta1})
D_trainer = gluon.Trainer(d_net.collect_params(), 'Adam', {'learning_rate': opt.lr,'beta1':opt.beta1})



## The learning loop

In [ ]:
g_net.collect_params().zero_grad()
d_net.collect_params().zero_grad()
counter = 0
for epoch in range(opt.niter):
    for i, (d, _ ) in enumerate(train_data):
        # update D
        data = d.as_in_context(opt.ctx)
        label = nd.ones(opt.batchSize, opt.ctx)
        with autograd.record():            
            output = d_net(data)
            D_error = nd.mean(binary_cross_entropy(output,label))
            D_x = nd.mean(output)

        D_error.backward()
        D_trainer.step(data.shape[0])
        
        noise = mx.ndarray.normal(loc = 0, scale = 1, shape = (opt.batchSize, opt.nz,1,1),ctx = opt.ctx)
        fake_image = g_net(noise)
        label = nd.zeros(opt.batchSize, opt.ctx)

        with autograd.record():
            output = d_net(fake_image)
            D_error_fake_image = nd.mean(binary_cross_entropy(output,label))
            D_G_z1 = nd.mean(output)

        D_error_fake_image.backward()
        D_trainer.step(fake_image.shape[0])
            
        #update G
        label = nd.ones(opt.batchSize, opt.ctx)
        with autograd.record():
            fake_image = g_net(noise)
            print(fake_image)
            output = d_net(fake_image)
            G_error = nd.mean(binary_cross_entropy(output,label))
            D_G_z2 = nd.mean(output)
        G_error.backward()
        G_trainer.step(fake_image.shape[0])
        
        
        print('[%d/%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f / %.4f'
              % (epoch,counter, opt.niter, i, len(train_data),
                 D_error.asnumpy(), G_error.asnumpy(), D_x.asnumpy(), D_G_z1.asnumpy(), D_G_z2.asnumpy()))
        trunctate = 1
        if i % (10*trunctate) == 0:
            #image_show(data)
            image_show(fake_image)
        if i % (100*trunctate) == 0:   
            filenameG = '%s/%s/G_Net_epoch_%d_%d' % (opt.outf, opt.dataset, epoch,counter)
            filenameD = '%s/%s/D_Net_epoch_%d_%d' % (opt.outf, opt.dataset, epoch,counter)
            g_net.save_params(filenameG)
            d_net.save_params(filenameD)
            counter = counter + 1
    counter = 0
